# Python Debugging for Jupyter Kernels

The below steps will allow you to debug a Juypter kernel written in Python with Visual Studio Code and the Python extension. Jupyter kernel code is indirectly invoked by the jupyter command line tool. Therefore, you cannot launch and debug a Jupyter kernel module directly from VS Code with a launch configuration. Instead, you need to make slight modifications to your Jupyter kernel code to allow a Python debugger to attach to it.

## Prerequisites (for the Jupyter Kernel example)

1. Clone the Jupyter PowerShell kernel to your local machine from [https://github.com/vors/jupyter-powershell](https://github.com/vors/jupyter-powershell).
2. Ensure that you have `jupyter` and `debugpy` installed:
    ```
    python -m pip install jupyter debugpy
    ```
    

## Configuring debugging

1. In your `launch.json` file (located in .vscode/launch.json), add the following configuration:
    
    ```
    {
     "version": "0.2.0",
     "configurations": [
         {
             "name": "Python: Attach",
             "type": "python",
             "request": "attach",
             "connect": {
                 "port": 5678
             }
         }
     ]
    }
    ```
    
2. In your kernel source code add the following code and save your changes - (within the jupyter-powershell project, you can add this in `kernel.py` in the `do_execute` function for example) :
    

```
# Add this at the top of your file
import debugpy 

# Add this at the first spot you'd like to start debugging from
# 5678 is the default attach port in the VS Code debug configurations. Unless a host and port are specified, host defaults to 127.0.0.1
debugpy.listen(5678) # ensure that this port is the same as the one in your launch.json
print("Waiting for debugger attach")
debugpy.wait_for_client()
debugpy.breakpoint()
print('break on this line')
```

3. Install the jupyter-powershell module with the changes from step 2 locally, by running the following command in the integrated terminal from the root of the jupyter-powershell repository:
    
    ```
    python -m powershell_kernel.install
    ```
    
4. Launch Jupyter by running the following command in the integrated terminal:
    
    ```
    jupyter notebook
    ```
    
5. Open a new Jupyter notebook and select the PowerShell kernel. Add some code to a cell and run it. If you added the code in the `do_execute` method as described in step 2, you should see the cell appear to stall with a busy indicator. This is because Jupyter is paused on the call to `debugpy.wait_for_client()`, that is, it is waiting for the debugger to connect to it.
    

![image](https://user-images.githubusercontent.com/30305945/93416305-64328900-f85a-11ea-8b37-9fb164806c93.png)

6. In VS Code, run the debugger `Python: Remote Attach` debug configuration. You should now see the variables from your program show up in the variable panel on the left, as shown in the screenshot below. Thereafter, you can set breakpoints interactively and step through your program as usual.

![image](https://user-images.githubusercontent.com/30305945/93418644-af9b6600-f85f-11ea-97d4-d4afa8b1a54a.png)